In [ ]:
from __future__ import unicode_literals

import sys, os
BIN = os.path.expanduser("../../../")
sys.path.append(BIN)

import numpy as np
from scipy.constants import m_p, c, e, pi
import matplotlib.pyplot as plt
%matplotlib inline

import copy
import itertools

from test_tools import generate_objects, BunchTracker, track, compare_traces, compare_projections, Machine

from PyHEADTAIL.feedback.feedback import OneboxFeedback, Kicker, PickUp
from PyHEADTAIL.feedback.processors.multiplication import ChargeWeighter
from PyHEADTAIL.feedback.processors.linear_transform import Averager
from PyHEADTAIL.feedback.processors.misc import Bypass
from PyHEADTAIL.feedback.processors.register import Register, TurnDelay, UncorrectedDelay

np.random.seed(0)

In [ ]:
""" 
    In this example a more complex feedback system, which is an ideal bunch feedback consisting of
    a separate pickup and a kicker is demonstrated. The details of the ideal bunch feedback can be found
    from the previous filr (001_ideal_feedbacks.ipynb).

"""

In [ ]:
# Basic parameters for the simulation
n_macroparticles = 10000
n_slices = 20
n_segments = 5
n_sigma_z = 3

n_turns = 50

# Longitudinal motion of the bunch is not taken into account in this example.
machine = Machine(n_segments= n_segments)
# The longitudinal motion can be included to simulations by uncommenting the following line
# machine = Machine(Q_s = 0.0020443)

bunch_ref, slicer_ref,trans_map, long_map = generate_objects(machine, n_macroparticles, 
                                                             n_slices,n_sigma_z)


In [ ]:
# This creates an artificially kicked bunch, which will be damped by using different feedback systems

slice_set = bunch_ref.get_slices(slicer_ref, statistics=True)
p_idx = slice_set.particles_within_cuts
s_idx = slice_set.slice_index_of_particle.take(p_idx)

# random kicks
kick_x = 0.003*(-1.0+2*np.random.rand(n_slices))
kick_y = 0.003*(-1.0+2*np.random.rand(n_slices))

for p_id, s_id in itertools.izip(p_idx,s_idx):
    bunch_ref.x[p_id] += kick_x[s_id]
    bunch_ref.y[p_id] += kick_y[s_id]
    
bunch_init = copy.deepcopy(bunch_ref)
tracker_ref = BunchTracker(bunch_init)
maps_ref = [i for i in trans_map]
# + [long_map]
track(n_turns, bunch_init,maps_ref,tracker_ref)

In [ ]:
# In the used code, the parameter 'gain' is determined as a fraction of the signal which is bypassed
# perfectly throught the signal processors

# There are two ways to set a gain value. If only one value is given, same value is used both in 
# horizontal and vertical planes. If two values are given, separed values (x,y) are used for horizontal and
# vertical planes.

feedback_gain = 0.1
# feedback_gain = (0.1,0.4)

# Parameters for the registers
delay = 0
n_values = 2

In [ ]:
# A reference feedback system, which is originally introduced in the file 001_ideal_feedbacks.ipynb:

bunch_OneBox_bunch = copy.deepcopy(bunch_ref)
tracker_OneBox_bunch = BunchTracker(bunch_OneBox_bunch)
slicer_OneBox_bunch = copy.deepcopy(slicer_ref)

processors_bunch_x = [
    ChargeWeighter(normalization = 'segment_average'),
    Averager()
]
processors_bunch_y = [
    ChargeWeighter(normalization = 'segment_average'),
    Averager()
]

feedback_map = OneboxFeedback(feedback_gain,slicer_OneBox_bunch,processors_bunch_x,processors_bunch_y)
total_map_OneBox_bunch = [i for i in trans_map] + [feedback_map]
#  + [long_map]

track(n_turns, bunch_OneBox_bunch,total_map_OneBox_bunch,tracker_OneBox_bunch)

In [ ]:
bunch_register_example = copy.deepcopy(bunch_ref)
tracker_register_example = BunchTracker(bunch_register_example)
slicer_register_example = copy.deepcopy(slicer_ref)

# Another reference system, which uses combiners to the betatron phase correction

processors_register_example_x = [
    ChargeWeighter(normalization = 'segment_average'),
    Averager(),
    TurnDelay(delay, machine.Q_x, n_values,additional_phase_advance=0.)
]
processors_register_example_y = [
    ChargeWeighter(normalization = 'segment_average'),
    Averager(),
    TurnDelay(delay, machine.Q_y, n_values,additional_phase_advance=0.)
]

feedback_map = OneboxFeedback(feedback_gain,slicer_register_example,
                              processors_register_example_x,processors_register_example_y)
total_map_register_example = [feedback_map] + [i for i in trans_map]
#  + [long_map]

track(n_turns, bunch_register_example,total_map_register_example,tracker_register_example)


In [ ]:
bunch_rotation_example = copy.deepcopy(bunch_ref)
tracker_rotation_example = BunchTracker(bunch_rotation_example)
slicer_rotation_example = copy.deepcopy(slicer_ref)

# An another way to perform a (artificial or ideal) betatron phase correction is to rotate signal directly when it is 
# riden. The rotation angle can be se by giving parameters phase_x and phase_y to an OneboxFeedback or a PickUp object.
# The signal can be delayed without corrections by using an UncorrectedDelay object.


processors_rotation_example_x = [
    ChargeWeighter(normalization = 'segment_average'),
    Averager(),
    UncorrectedDelay(delay)
]
processors_rotation_example_y = [
    ChargeWeighter(normalization = 'segment_average'),
    Averager(),
    UncorrectedDelay(delay)
]

phase_rotation_x = 0.
phase_rotation_y = 0.

# rotation required by one turn delay
phase_rotation_x += delay * 2. * pi * machine.Q_x
phase_rotation_y += delay * 2. * pi * machine.Q_y

# rotation required by transformation from pickup (displacement) to kicker (divergence)
phase_rotation_x += pi/2.
phase_rotation_y += pi/2.

# phase_rotation_x = None
# phase_rotation_y = None

feedback_map = OneboxFeedback(feedback_gain,slicer_rotation_example,
                              processors_rotation_example_x,processors_rotation_example_y,
                              phase_x=phase_rotation_x, phase_y=phase_rotation_y,
                              beta_x=machine.beta_x, beta_y=machine.beta_y,
                              pickup_axis='displacement', kicker_axis='divergence')
total_map_rotation_example = [feedback_map] + [i for i in trans_map]
#  + [long_map]

track(n_turns, bunch_rotation_example,total_map_rotation_example,tracker_rotation_example)

In [ ]:
bunch_separated_example = copy.deepcopy(bunch_ref)
tracker_separated_example = BunchTracker(bunch_separated_example)
slicer_separated_example = copy.deepcopy(slicer_ref)

# If the feedback system consists of only one pickup and one kicker, the betatron correction between the pickup and kicker
# can also be (artificially or ideally) performed by a phase rotation on the pickup. 

pickup_beta_x = machine.beta_x
pickup_beta_y = machine.beta_y

kicker_beta_x = machine.beta_x
kicker_beta_y = machine.beta_y

# The total betatron phase advance over the accelerator is 2*pi*Q. In this example, the accelerator is divided into 
# n segments. Thus, the segment length in the betatron phase is 2*pi*Q/n_segments. The location of the pickup is chosen 
# to be after the first segment and the location of the kicker after the second segment, i.e. 

pickup_location_x = 1.*2.*pi/float(n_segments)*machine.Q_x
pickup_location_y = 1.*2.*pi/float(n_segments)*machine.Q_y

kicker_location_x = 2.*2.*pi/float(n_segments)*machine.Q_x
kicker_location_y = 2.*2.*pi/float(n_segments)*machine.Q_y

# Exactly same signal processors as in the previous example are used in the pickup
processors_pickup_x = [
    ChargeWeighter(normalization = 'segment_average'),
    Averager(),
    Register(1, machine.Q_x, delay)
]
processors_pickup_y = [
    ChargeWeighter(normalization = 'segment_average'),
    Averager(),
    Register(1, machine.Q_y, delay)
]

phase_rotation_x = 0.
phase_rotation_y = 0.

# rotation required by one turn delay
phase_rotation_x +=  (delay) * 2. * pi * machine.Q_x
phase_rotation_y +=  (delay) * 2. * pi * machine.Q_y

# rotation required by transformation from pickup (displacement) to kicker (divergence)
phase_rotation_x += pi/2.
phase_rotation_y += pi/2.

# rotation required by the location difference between the pickup and the kicker
phase_rotation_x += kicker_location_x - pickup_location_x
phase_rotation_y += kicker_location_y - pickup_location_y

# The map element is created by giving a slicer object, the signal processors and the beam parameters as input parameters
pickup_map = PickUp(slicer_separated_example,processors_pickup_x,processors_pickup_y, pickup_location_x, pickup_beta_x,
                 pickup_location_y, pickup_beta_y, phase_x=phase_rotation_x, phase_y=phase_rotation_y,)

# The kicker can be created very similarly to the pickup. In addition to the PickUp,
# a gain value and a list of register references are given to the kicker

# In this example, the signals are not modified in the kicker, i.e. only bypass processors are used
processors_kicker_x = [Bypass()]
processors_kicker_y = [Bypass()]

# A list of references to registers from the signal processors(s) of the pickup(s)
registers_x = [processors_pickup_x[-1]]
registers_y = [processors_pickup_y[-1]]

kicker_map = Kicker(feedback_gain, slicer_separated_example, processors_kicker_x, processors_kicker_y,
                    registers_x, registers_y, kicker_location_x, kicker_beta_x, kicker_location_y, kicker_beta_y,
                    combiner='dummy')

# Finally the maps must be added to correct slots of the total map determined by the betaton phase advance locations of 
# the picup and the kicker
total_map_separated_example = [trans_map[0]] + [pickup_map] + [trans_map[1]] + [kicker_map]
for element in trans_map[2:]:
    total_map_separated_example += [element]
# total_map_separated_example += [long_map]
    
    
track(n_turns, bunch_separated_example,total_map_separated_example,tracker_separated_example)

In [ ]:
# In here, the traces and the projections from different implementations of the feedback system are compared.
# Note the scale in the emittance figures.

compare_traces([tracker_OneBox_bunch,tracker_register_example,tracker_rotation_example,tracker_separated_example],
               ['Ideal', 'Delayed', 'Rotated', 'Separated'])
compare_projections([ bunch_OneBox_bunch,  bunch_register_example,  bunch_rotation_example, bunch_separated_example], 
                    ['Ideal', 'Delayed', 'Rotated', 'Separated'])

# compare_traces([tracker_OneBox_bunch,tracker_register_example],
#                ['Ideal', 'Delayed', 'Separated'])
# compare_projections([ bunch_OneBox_bunch,  bunch_register_example], 
#                     ['Ideal', 'Delayed', 'Separated'])

Jani Komppula, CERN, 2017